以下ではWebサービスの予測エンジンとして、機械学習のアプリケーションを組み込む方法を説明する。<br>
<ul>
<li>トレーニングされた機械学習モデルの現在の状態を保存</li>
<li>データストレージとしてSQLiteデータベースを使用</li>
<li>人気の高いWebフレームワークFlaskを使ってWebアプリケーションを開発</li>
<li>機械学習アプリケーションをパブリックWebサーバーにデプロイ（展開）する</li>
</ul>

## <font color='blue'>学習済みのscikit-learn推定器をシリアライズする</font>

pickleを用いることで、再学習することなく学習済みのモデルを用いることができる。<br>

下記のvectorizer.pyを用いてデシリアライズ化できるか確認

In [2]:
%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Overwriting movieclassifier/vectorizer.py


<font color='red'>以下のコードで作業ディレクトリの変更を行う</font>

In [11]:
# このコードがないと次のコードのvectorizerが動作しないということに注意
import os
os.chdir('movieclassifier')

FileNotFoundError: [Errno 2] No such file or directory: 'movieclassifier'

In [13]:
import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [16]:
import numpy as np
label = {0:'negative', 1:'positive'}

example = ['I love this movie']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], clf.predict_proba(X).max()*100))

Prediction: positive
Probability: 82.52%


## <font color='blue'>データストレージとしてSQLiteデータベースをセットアップする</font>

SQLiteは小さなプロジェクトや単純なWebアプリケーションにうってつけ

以下のコードにより、movieclassifierディレクトリに新しいSQLiteデータベースを作成し、2つのサンプル映画レビューを格納

In [18]:
import sqlite3
import os

if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

<font color='red'>SQLiteの注意点<br></font>
<ul>
<li>SQLiteには既存のテーブルを置き換える機能が実装されていない</li>
</ul>

In [19]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2015-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()
print(results)

[('I love this movie', 1, '2017-05-25 03:07:24'), ('I disliked this movie', 0, '2017-05-25 03:07:24')]


## <font color='blue'>Flaskを使ってWebアプリケーションを開発する</font>